Description:

Ce notebook est utile pour afficher les courbes de tendance de SoH par modèle (pour l'instant tesla) et on peut même aller plus loin avec les courbe de tendance de SoH pour les versions (ou tesla code).

Il y'a la possibilité d'ajouter les intervals de confiance de ces courbes. 

Next setps:
- forcer le passage en 0,1 -> mieux que ce qui est fait (solution ittérative)

In [ ]:
import pandas as pd
from core.sql_utils import *
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit

In [ ]:
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT * FROM vehicle_data vd
            join vehicle v
            on v.id = vd.vehicle_id
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            WHERE vm.model_name like '%model%';"""), con)



dbeaver_df.head()
dbeaver_df['timestamp'] = pd.to_datetime(dbeaver_df['timestamp'])
dbeaver_df.sort_values("timestamp", inplace=True)

In [ ]:
# on drop MTY13/MT336/MT337
dbeaver_filtered = dbeaver_df[~dbeaver_df['version'].isin(['MT336','MT337', 'MTY13'])][['version', 'soh', 'odometer', 'model_name', 'type']].copy().dropna()


In [ ]:
px.scatter(dbeaver_filtered[['version', 'soh', 'odometer']].dropna(),  x='odometer', y='soh', color="version")

## Graphs des tendances 

faire avec itteration 

In [ ]:
# Fonction logarithmique pour la tendance
def log_function(x, a):
    return 1 + a * np.log1p(x/1000) 

In [ ]:

fig_model = go.Figure()
model_colors = {model: px.colors.qualitative.Plotly[i] for i, model in enumerate(dbeaver_filtered.model_name.unique())}

for model in dbeaver_filtered.model_name.unique():
    
    df_model_temp = dbeaver_filtered[dbeaver_filtered['model_name']==model].sort_values('odometer').copy()
    popt, covariance = curve_fit(log_function, df_model_temp['odometer'], 
                        df_model_temp['soh'])
    x_vals = np.linspace(0.1,  df_model_temp.odometer.max(), 500)
    y_vals = log_function(x_vals, *popt)
    perr = np.sqrt(np.diag(covariance))
    delta = 1.96 * np.sqrt((perr[0] * np.log(x_vals))**2)  # Intervalle à 95%

    color = model_colors[model]  # Couleur unique pour le modèle
    color_rgba = color.replace('rgb', 'rgba').replace(')', ', 0.2)')  # Version transparente

    # Génération des valeurs ajustées
    fig_model.add_traces(go.Scatter(x=x_vals, y=y_vals, name=f'{model} trend', line=dict(color=color)))
### Décommenter en dessous si on veut les intervals de confiance associé.      
    
    fig_model.add_trace(go.Scatter(
    x=np.concatenate([x_vals, x_vals[::-1]]), 
    y=np.concatenate([y_vals - delta, (y_vals + delta)[::-1]]),
    fill='toself',
    fillcolor=color_rgba,  
    showlegend=True,
    name="Intervalle de confiance 95%",
    opacity=.5
))


### Décommenter en dessous si on veut les points de SoH 
    # fig_model.add_traces(go.Scatter(x=df_model_temp['odometer'].values, y=df_model_temp['soh'], name=f'{model} value', mode='markers'))
    
    fig_model.update_xaxes(title='odometer')
    fig_model.update_yaxes(title='soh')
    fig_model.update_layout(title="Courbe de tendance pour les modèles tesla")

In [ ]:
# définition des différentes figures 
fig_version_3 = go.Figure()
fig_version_X = go.Figure()
fig_version_S = go.Figure()
fig_version_Y = go.Figure()

# courbe pour les tesla_code des model 3 
for version in dbeaver_filtered[dbeaver_filtered['model_name'] == "model 3"].type.unique():
    df_temp =  dbeaver_filtered[dbeaver_filtered['type']==version]
    if len(df_temp) > 0:
        popt, _ = curve_fit(log_function, 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['odometer'], 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['soh'], )
        
        x_vals = np.linspace(0.1,  
                            max(150000, max(dbeaver_filtered[dbeaver_filtered['type']==version].odometer)), 
                            500)
        y_vals = log_function(x_vals, *popt)

        fig_version_3.add_traces(go.Scatter(x=x_vals, y=y_vals, name=f'{version} trend'))
        fig_version_3.update_xaxes(title='odometer')
        fig_version_3.update_yaxes(title='soh')
        fig_version_3.update_layout(title="Courbe de tendance pour les tesla code model 3")
        # fig_version.add_traces(go.Scatter(x=dbeaver_filtered[dbeaver_filtered['type']==version]['odometer'].values, 
        #                                   y=dbeaver_filtered[dbeaver_filtered['type']==version]['soh'], 
        #                                   name=f'{version} value', mode='markers'))
        
# courbe pour les tesla_code des model x 
for version in dbeaver_filtered[dbeaver_filtered['model_name'] == "model x"].type.unique():
    df_temp =  dbeaver_filtered[dbeaver_filtered['type']==version]
    if len(df_temp) > 0:
        popt, _ = curve_fit(log_function, 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['odometer'], 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['soh'], )
        
        x_vals = np.linspace(0.1,  
                            max(150000, max(dbeaver_filtered[dbeaver_filtered['type']==version].odometer)), 
                            500)
        y_vals = log_function(x_vals, *popt)

        fig_version_X.add_traces(go.Scatter(x=x_vals, y=y_vals, name=f'{version} trend'))
        fig_version_X.update_xaxes(title='odometer')
        fig_version_X.update_yaxes(title='soh')
        fig_version_X.update_layout(title="Courbe de tendance pour les tesla code model X")
            
# courbe pour les tesla_code des model y

for version in dbeaver_filtered[dbeaver_filtered['model_name'] == "model y"].type.unique():
    df_temp =  dbeaver_filtered[dbeaver_filtered['type']==version]
    if len(df_temp) > 0:
        popt, _ = curve_fit(log_function, 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['odometer'], 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['soh'], )
        
        x_vals = np.linspace(0.1,  
                            max(150000, max(dbeaver_filtered[dbeaver_filtered['type']==version].odometer)), 
                            500)
        y_vals = log_function(x_vals, *popt)

        fig_version_Y.add_traces(go.Scatter(x=x_vals, y=y_vals, name=f'{version} trend'))
        fig_version_Y.update_xaxes(title='odometer')
        fig_version_Y.update_yaxes(title='soh')
        fig_version_Y.update_layout(title="Courbe de tendance pour les tesla code model Y")
        

# courbe pour les tesla_code des model s

for version in dbeaver_filtered[dbeaver_filtered['model_name'] == "model s"].type.unique():
    print(version)
    df_temp =  dbeaver_filtered[dbeaver_filtered['type']==version]
    if len(df_temp) > 0:

        popt, _ = curve_fit(log_function, 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['odometer'], 
                            dbeaver_filtered[dbeaver_filtered['type']==version]['soh'], )
        
        x_vals = np.linspace(0.1,  
                            max(150000, max(dbeaver_filtered[dbeaver_filtered['type']==version].odometer)), 
                            500)
        y_vals = log_function(x_vals, *popt)

        fig_version_S.add_traces(go.Scatter(x=x_vals, y=y_vals, name=f'{version} trend'))
        fig_version_S.update_xaxes(title='odometer')
        fig_version_S.update_yaxes(title='soh')
        fig_version_S.update_layout(title="Courbe de tendance pour les tesla code model S")

In [ ]:
fig_model.show()

In [ ]:
fig_version_X.show()

In [ ]:
fig_version_X.show()

In [ ]:
fig_version_S.show()

In [ ]:
fig_version_3.show()

In [ ]:
fig_version_Y.show()